### **Búsqueda y Minería de Información 2022-23**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Motores de búsqueda e indexación**

Fechas:

* Comienzo: martes 21 / jueves 23 de febrero
* Entrega: martes 28 / jueves 30 de marzo (14:00)

## Autores

Guillermo Martín-Coello & Daniel Varela

# Introducción

## Objetivos

Los objetivos de esta práctica son:

* La implementación eficiente de funciones de ránking, particularizada en el modelo vectorial.
*	La implementación de índices eficientes para motores de búsqueda. 
*	La implementación de un método de búsqueda proximal.
*	La dotación de estructuras de índice posicional que soporten la búsqueda proximal.
*	La implementación del algoritmo PageRank.

Se desarrollarán implementaciones de índices utilizando un diccionario y listas de postings. Y se implementará el modelo vectorial utilizando estas estructuras más eficientes para la ejecución de consultas.

Los ejercicios básicos consistirán en la implementación de algoritmos y técnicas estudiados en las clases de teoría, con algunas propuestas de extensión opcionales. Se podrá comparar el rendimiento de las diferentes versiones de índices y buscadores, contrastando la coherencia con los planteamientos estudiados a nivel teórico.

Mediante el nivel de abstracción seguido, se conseguirán versiones intercambiables de índices y buscadores. El **único buscador que no será intercambiable es el de Whoosh**, que sólo funcionará con sus propios índices.

## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado. No obstante, aquellos ejercicios marcados con un asterisco (*) tienen una complejidad un poco superior a los demás (que suman 7.5 puntos), y permiten, si se realizan todos, una nota superior a 10. 

El peso de la nota de esta práctica en la calificación final de prácticas es del **40%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. La calidad se valorará por los **resultados** conseguidos (economía de consumo de RAM, disco y tiempo; tamaño de las colecciones que se consigan indexar) pero también del **mérito** en términos del interés de las técnicas aplicadas y la buena programación.

La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto añadiendo los módulos entregados por el estudiante a los módulos facilitados en la práctica, ejecutando la *celda de prueba* así como otros tests adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**. Si se necesita entregar algún fichero adicional (por ejemplo, imágenes) se puede subir un fichero ZIP a la tarea correspondiente de Moodle. En cualquiera de los dos casos, el nombre del fichero a subir será **bmi-p2-XX**, donde XX debe sustituirse por el número de pareja (01, 02, ..., 10, ...).

## Indicaciones

Se sugiere trabajar en la práctica de manera incremental, asegurando la implementación de soluciones sencillas y mejorándolas de forma modular (la propia estructura de ejercicios plantea ya esta forma de trabajar).

Se podrán definir clases o módulos adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: la **celda de prueba** deberá ejecutar correctamente <ins>sin ninguna modificación</ins> (ten en cuenta que, aquellos ejercicios que no se hayan realizado, lanzan una excepción que se captura en dicha celda, por lo que no debería ser necesario modificarla).

Asimismo, se recomienda indexar sin ningún tipo de stopwords ni stemming, para poder hacer pruebas más fácilmente con ejemplos “de juguete”.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python a lo largo de este *notebook*, con las que el estudiante integrará las suyas propias. 
Las clases parten del código de la práctica anterior.
Igual que en la práctica 1, la **celda de prueba** (al final del enunciado) implementa un programa que deberá funcionar con las clases a implementar por el estudiante.
*	Las colecciones de prueba de la práctica 1: <ins>toys.zip</ins> (que se descomprime en dos carpetas toy1 y toy2), <ins>docs1k.zip</ins> con 1.000 documentos HTML y un pequeño fichero <ins>urls.txt</ins>. 
*	Una colección más grande: <ins>docs10k.zip</ins> con 10.000 documentos HTML.
*	Varios grafos para probar PageRank: <ins>graphs.zip</ins>.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la celda de prueba (salvo los tiempos de ejecución que pueden cambiar, aunque la tendencia en cuanto a qué métodos tardan más o menos debería mantenerse).

### Clases genéricas ya implementadas

En la siguiente celda de código, se encuentran ya implementadas las clases *Index* y *Builder* de manera que facilite la creación de otros índices a partir de las mismas. 

Estudia esta implementación y compara las **decisiones de diseño** tomadas con las vuestras en la práctica anterior.
Ten en cuenta que las funciones de TF e IDF están **sin implementar**.

In [91]:
import os, os.path
import re
import math
import pickle
import zipfile
from abc import ABC, abstractmethod
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl

class Config(object):
  # variables de clase
  NORMS_FILE = "docnorms.dat"
  PATHS_FILE = "docpaths.dat"
  INDEX_FILE = "serialindex.dat"
  DICTIONARY_FILE = "dictionary.dat"
  POSTINGS_FILE = "postings.dat"
  POSITIONS_FILE = "positions.dat"


class BasicParser:
    @staticmethod
    def parse(text):
        return re.findall(r"[^\W\d_]+|\d+", text.lower())

def tf(freq):
    """
    tf(freq)
        - Return the term frequency.
    @param freq: term frequency
    """
    return 1 + math.log2(freq) if freq > 0 else 0


def idf(df, n):
    """
    idf(df, n)
        - Return the inverse document frequency.
    @param df: document frequency
    @param n: number of documents
    """
    return math.log2((n + 1) / (df + 0.5))

"""
    This is an abstract class for the search engines
"""
class Searcher(ABC):
    def __init__(self, index, parser=BasicParser()):
        self.index = index
        self.parser = parser
    @abstractmethod
    def search(self, query, cutoff):
        """ Returns a list of documents encapsulated in a SearchRanking class """

class Index:
    def __init__(self, dir=None):
        self.docmap = []
        self.modulemap = {}
        if dir: self.open(dir)
    def add_doc(self, path):
        self.docmap.append(path)
    def doc_path(self, docid):
        return self.docmap[docid]
    def doc_module(self, docid):
        if docid in self.modulemap:
            return self.modulemap[docid]
        return None
    def ndocs(self):
        return len(self.docmap)
    def doc_freq(self, term):
        return len(self.postings(term))
    def term_freq(self, term, docID):
        post = self.postings(term)
        if post is None: return 0
        for posting in post:
            if posting[0] == docID:
                return posting[1]
        return 0
    def total_freq(self, term):
        freq = 0
        for posting in self.postings(term):
            freq += posting[1]
        return freq
    def postings(self, term):
        # used in more efficient implementations
        return list()
    def positional_postings(self, term):
        # used in positional implementations
        return list()
    def all_terms(self):
        return list()
    def save(self, dir):
        if not self.modulemap: self.compute_modules()
        p = os.path.join(dir, Config.NORMS_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.modulemap, f)        
    def open(self, dir):
        try:
            p = os.path.join(dir, Config.NORMS_FILE)
            with open(p, 'rb') as f:
                self.modulemap = pickle.load(f)
        except OSError:
            # the file may not exist the first time
            pass
    def compute_modules(self):
        for term in self.all_terms():
            idf_score = idf(self.doc_freq(term), self.ndocs())
            post = self.postings(term)
            if post is None: continue
            for docid, freq in post:
                if docid not in self.modulemap: self.modulemap[docid] = 0
                self.modulemap[docid] += math.pow(tf(freq) * idf_score, 2)
        for docid in range(self.ndocs()):
            self.modulemap[docid] = math.sqrt(self.modulemap[docid]) if docid in self.modulemap else 0

import shutil
class Builder:
    def __init__(self, dir, parser=BasicParser()):
        if os.path.exists(dir): shutil.rmtree(dir)
        os.makedirs(dir)
        self.parser = parser
    def build(self, path):
        if zipfile.is_zipfile(path):
            self.index_zip(path)
        elif os.path.isdir(path):
            self.index_dir(path)
        else:
            self.index_url_file(path)
    def index_zip(self, filename):
        file = zipfile.ZipFile(filename, mode='r', compression=zipfile.ZIP_DEFLATED)
        for name in sorted(file.namelist()):
            with file.open(name, "r", force_zip64=True) as f:
                self.index_document(name, BeautifulSoup(f.read().decode("utf-8"), "html.parser").text)
        file.close()
    def index_dir(self, dir):
        for subdir, dirs, files in os.walk(dir):
            for file in sorted(files):
                path = os.path.join(dir, file)
                with open(path, "r") as f:
                    self.index_document(path, f.read())
    def index_url_file(self, file):
        with open(file, "r") as f:
            self.index_urls(line.rstrip('\n') for line in f)
    def index_urls(self, urls):
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        for url in urls:
            self.index_document(url, BeautifulSoup(urlopen(url, context=ctx).read().decode("utf-8"), "html.parser").text)
    def index_document(self, path, text):
        raise NotImplementedError # to be implemented by child class
    def commit(self):
        raise NotImplementedError # to be implemented by child class

### Ejemplo de buscador

En la siguiente celda se encuentra una implementación de un buscador basado en coseno que es relativamente lento. En los siguientes ejercicios veremos formas de acelerar el proceso (sin cambiar los resultados).

In [92]:
# from previous lab
class SlowVSMSearcher(Searcher):
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        for docid in range(self.index.ndocs()):
            score = self.score(docid, qterms)
            if score:
                ranking.push(self.index.doc_path(docid), score)
        return ranking

    def score(self, docid, qterms):
        prod = 0
        for term in qterms:
            prod += tf(self.index.term_freq(term, docid)) \
                    * idf(self.index.doc_freq(term), self.index.ndocs())
        mod = self.index.doc_module(docid)
        if mod:
            return prod / mod
        return 0

### Clases Whoosh

En la siguiente celda podrás encontrar la adaptación a nuestras interfaces de los índices de Whoosh, en concreto, de tres variantes que permite usar la librería (observa los distintos Schema's usados y qué metodos se han reimplementado en cada caso).

In [93]:
try:
  import whoosh
except ModuleNotFoundError:
  !pip install whoosh
  import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser

# A schema in Whoosh is the set of possible fields in a document in the search space. 
# We just define a simple 'Document' schema, with a path (a URL or local pathname)
# and a content.
SimpleDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False))
ForwardDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False,vector=Format))
PositionalDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=True))

class WhooshBuilder(Builder):
    def __init__(self, dir, schema=SimpleDocument):
        super().__init__(dir)
        self.whoosh_writer = whoosh.index.create_in(dir, schema).writer(procs=1, limitmb=16384, multisegment=True)
        self.dir = dir

    def index_document(self, p, text):
        self.whoosh_writer.add_document(path=p, content=text)

    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshIndex(self.dir)
        index.save(self.dir)

class WhooshForwardBuilder(WhooshBuilder):
    def __init__(self, dir):
        super().__init__(dir, ForwardDocument)
    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshForwardIndex(self.dir)
        index.save(self.dir)

class WhooshPositionalBuilder(WhooshBuilder):
    def __init__(self, dir):
        super().__init__(dir, PositionalDocument)
    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshPositionalIndex(self.dir)
        index.save(self.dir)

class WhooshIndex(Index):
    def __init__(self, dir):
        super().__init__(dir)
        self.whoosh_reader = whoosh.index.open_dir(dir).reader()    
    def total_freq(self, term):
        return self.whoosh_reader.frequency("content", term)
    def doc_freq(self, term):
        return self.whoosh_reader.doc_frequency("content", term)
    def doc_path(self, docid):
        return self.whoosh_reader.stored_fields(docid)['path']
    def ndocs(self):
        return self.whoosh_reader.doc_count()
    def all_terms(self):
        return list(self.whoosh_reader.field_terms("content"))
    def postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("frequency") \
            if self.doc_freq(term) > 0 else []

class WhooshForwardIndex(WhooshIndex):
    def term_freq(self, term, docID) -> int:
        if self.whoosh_reader.has_vector(docID, "content"):
            v = self.whoosh_reader.vector(docID, "content")
            v.skip_to(term)
            if v.id() == term:
                return v.value_as("frequency")
        return 0

class WhooshPositionalIndex(WhooshIndex):
    def positional_postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("positions") \
            if self.doc_freq(term) > 0 else []

class WhooshSearcher(Searcher):
    def __init__(self, dir):
        self.whoosh_index = whoosh.index.open_dir(dir)
        self.whoosh_searcher = self.whoosh_index.searcher()
        self.qparser = QueryParser("content", schema=self.whoosh_index.schema)
    def search(self, query, cutoff):
        return map(lambda scoredoc: (self.doc_path(scoredoc[0]), scoredoc[1]),
                   self.whoosh_searcher.search(self.qparser.parse(query), limit=cutoff).items())
    def doc_path(self, docid):
        return self.whoosh_index.reader().stored_fields(docid)['path']

# Ejercicio 1: Implementación de un modelo vectorial eficiente

Se mejorará la implementación de la práctica anterior aplicando algoritmos estudiados en las clases de teoría. En particular, se utilizarán listas de postings en lugar de un índice forward.

La reimplementación seguirá haciendo uso de la clase abstracta Index, y se podrá probar con cualquier implementación de esta clase (tanto la implementación de índice sobre Whoosh como las propias). 

## Ejercicio 1.1: Método orientado a términos (3pt)

Escribir una clase TermBasedVSMSearcher que implemente el modelo vectorial coseno por el método orientado a términos.

In [94]:
"""
Class to create a Term-Based Vector Space Model Searcher
"""
class TermBasedVSMSearcher(Searcher):
    """
    Constructor
    Parameters:
        - index: the index for the search
        - parser: the parser for the search
    """
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    """
    Search method to find a query in the index
    Parameters:
        - query: the query to be searched
        - cutoff: number of results to be returned (maximum)
    Return:
        - SearchRanking with results
    """
    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        postings = {}
        for term in qterms:
            for post in self.index.postings(term):
                if post[0] not in postings.keys():
                    postings[post[0]]=0
                postings[post[0]] += self.score(post[0], [term])
        for doc_id in postings.keys():
            if doc_id in postings.keys():
                ranking.push(self.index.doc_path(doc_id), postings[doc_id])

        return ranking

    """
    Score method to compute the score of a query in a document
    Parameters:
        - docid: the document id
        - qterms: the query terms on a list
    Return:
        - the score of the query in the document
    """
    def score(self, docid, qterms):
        prod = 0
        for term in qterms:
            prod += tf(self.index.term_freq(term, docid)) \
                    * idf(self.index.doc_freq(term), self.index.ndocs())
        mod = self.index.doc_module(docid)
        if mod:
            return prod / mod
        return 0

### Explicación/documentación

Para implementar el modelo vectorial con coseno por el método orientado a términos hemos trabajado a partir de la clase abstracta Searcher de la que hemos heredado. Los dos cambios necesarios han sido los siguientes:

Para el método search hemos tenido que reescribir su funcionamiento para que utilize el algoritmo del coseno. Éste simplemente obtendrá como argumento los elementos de la query, sobre los que iterará, y para cada uno comprobará los postings guardados en el índice e irá encontrando el score correspondiente de cada documento gracias al método "score". los scores obtenidos los añadirá en un ranking para obtener los n primeros, siendo n el cutoff entrado como argumento.

Para poder implementar la funcionalidad del método anterior también necesitamos reescribir el método score, que utilizará el algorimo del coseno para calcular el valor de cada palabra en función de su importancia en el documento (tf) y de su valor discriminatorio respecto a todos los documentos (idf). Así, se obtiene el score que utiliza el método search.

## Ejercicio 1.2: Método orientado a documentos* (1pt)

Implementar el método orientado a documentos (con heap de postings) en una clase DocBasedVSMSearcher.

In [95]:
import heapq
"""
Class to create a Document-Based Vector Space Model Searcher
"""
class DocBasedVSMSearcher(Searcher):
    """
    Constructor
    Parameters:
        - index: the index for the search
        - parser: the parser for the search
    """
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)
        

    """
    Search method to find a query in the index
    Parameters:
        - query: the query to be searched
        - cutoff: number of results to be returned (maximum)
    Return:
        - SearchRanking with results
    """
    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        postings = []

        # Extract all postings from the query terms: postings = (docid, score)
        for term in qterms:
            for post in self.index.postings(term):
                postings.append((post[0], self.score(post[0], post[1], term)))
        
        # Create new  min heap for postings
        heap = postings
        heapq.heapify(heap)
        if len(heap) <= 0:
            print("Error, not enough elements in heap")
            return ranking
        prev, score = heapq.heappop(heap)
        heapq.heapify(heap)
        while len(heap) > 0:
            curr, aux = heapq.heappop(heap)
            if prev == curr:
                score += aux
            else:
                mod = self.index.doc_module(prev)
                if mod != 0:
                    ranking.push(self.index.doc_path(prev), score/mod)
                else:
                    
                    ranking.push(self.index.doc_path(prev), 0)
                prev = curr
                score = aux
                
            heapq.heapify(heap)
        mod = self.index.doc_module(curr)
        if mod != 0:
            ranking.push(self.index.doc_path(curr), score/mod)
        else:
            
            ranking.push(self.index.doc_path(curr), 0)

        return ranking

    """
    Score method to compute the score of a query in a document
    Parameters:
        - docid: the document id
        - qterms: the query terms on a list
    Return:
        - the score of the query in the document
    """
    def score(self, docid, frec, term):
        prod = tf(frec) \
                * idf(self.index.doc_freq(term), self.index.ndocs())

        return prod

### Explicación/documentación

Para implementar el método orientado a documentos hemos seguido el razonamiento del algoritmo utilizando un heap de postings. Esto consiste en que para cada término de la query extraemos los postings y utilizamos un heap para sacarlos en el orden correcto. Para simplificar el código y evitar repetir llamadas hemos calculado el score de cada elemento directamente en su posting, en vez de guardar la frecuencia y calcularlo más adelante. Cuando se extrae del heap un posting, se comprueba su documento, y, si es diferente al anterior se comienza a guardar el score del nuevo documento, mientras que si es el mismo documento, se suma el score al previamente obtenido en anteriores iteraciones. Mantenemos un último push fuera del bucle para que se añada el último documento.

## Ejercicio 1.3: Heap de ránking (0.5pt)

Reimplementar la clase entregada SearchRanking para utilizar un heap de ránking (se recomienda usar el módulo [heapq](https://docs.python.org/3/library/heapq.html)), es decir, que permita almacenar un **número limitado de documentos** en memoria y su puntuación asociada. 

Nótese que esta opción se aprovecha mejor con la implementación orientada a documentos, aunque es compatible con la orientada a términos.

In [96]:
import heapq
"""
Class to create a Search Ranking
"""
class SearchRanking:
    """
    Constructor
    Parameters:
        - cutoff: number of results to be returned (maximum)
    """
    def __init__(self, cutoff):
        self.cutoff = cutoff
        self.ranking = [] 

    """
    Push method to add a new result to the ranking
    Parameters:
        - docid: the document id
        - score: the score of the document
    """
    def push(self, docid, score):
        if (len(self.ranking)<self.cutoff):
            heapq.heappush(self.ranking, (docid, score))
        else:
            heapq.heappushpop(self.ranking, (docid, score))
    
    """
    Iterator method to iterate over the ranking
    """
    def __iter__(self):
        min_l = min(len(self.ranking), self.cutoff)
        ## sort ranking
        self.ranking.sort(key=lambda tup: tup[1], reverse=True)
        return iter(self.ranking[0:min_l])

### Explicación/documentación

Para implementar la clase Search Ranking hemos utilizado la librería heapq, que nos aporta métodos para hacer un heap de manera rápida y eficiente. El código consiste en un método push que añade un elemento al heap, y en el caso de que el número de elementos sea mayor que el cutoff, realiza un pop para eliminar el elemento de menor valor. Esto permite que en el heap siempre queden los n elementos con mayor valor.

# Ejercicio 2: Índice en RAM (3pt)

Implementar un índice propio que pueda hacer las mismas funciones que la implementación basada en Whoosh definida en la práctica 1. Como primera fase más sencilla, los índices se crearán completamente en RAM. Se guardarán a disco y leerán de disco en modo serializado (ver módulo [pickle](https://docs.python.org/3/library/pickle.html)).

Para guardar el índice se utilizarán los nombres de fichero definidos por las variables estáticas de la clase Config. 

Antes de guardar el índice, se borrarán todos los ficheros que pueda haber creados en el directorio del índice. Asimismo, el directorio se creará si no estuviera creado, de forma que no haga falta crearlo a mano. Este detalle se hará igual en los siguientes ejercicios.

## Ejercicio 2.1: Estructura de índice

Implementar la clase RAMIndex como subclase de Index con las estructuras necesarias: diccionario, listas de postings, más la información que se necesite. 

Para este ejercicio en las listas de postings sólo será necesario guardar los docIDs y las frecuencias; no es necesario almacenar las posiciones de los términos.

In [97]:
"""
Class to create an Index based on RAM
"""
class RAMIndex(Index):
    """
    Constructor
    Parameters:
        - dir: index directory
    """
    def __init__(self, dir = None):
        self.docmap = []
        self.modulemap = {}
        self.builder_postings = {}
        self.loaded_postings = {}
        if dir: self.open(dir)
            
    """
    Returns all postings for a term
    Parameters:
        - term: the term to be searched
    Return:
        - list of tuples (docid, freq)
    """
    def postings(self, term):
        return self.loaded_postings[term]
    
    """
    Returns the terms in the index
    Return:
        - list of terms
    """
    def all_terms(self):
        return self.loaded_postings.keys()

    """
    Saves the builded postings created by the index builder.
    Builds the loaded_postings dictionary, needed for the search.
    Parameters:
        - dir: the directory where the index will be saved
    """
    def save(self, dir):
        super().save(dir)
        self.loaded_postings = {}
        for term in self.builder_postings.keys():
            self.loaded_postings[term] = []
            for doc_id in self.builder_postings[term].keys():
                self.loaded_postings[term].append((doc_id, self.builder_postings[term][doc_id]))
        p = os.path.join(dir, Config.POSTINGS_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.loaded_postings, f) 

        p2 = os.path.join(dir, Config.PATHS_FILE)    
        with open(p2, 'wb') as f:
            pickle.dump(self.docmap, f)
    
    """
    Loads the index from a directory
    Parameters:
        - dir: the directory where the index is saved
    """
    def open(self, dir):
        super().open(dir)    
        try:
            p = os.path.join(dir, Config.POSTINGS_FILE)
            with open(p, 'rb') as f:
                self.loaded_postings = pickle.load(f)
            p2 = os.path.join(dir, Config.PATHS_FILE)         
            with open(p2, 'rb') as f:
                self.docmap = pickle.load(f)

        except OSError:
            print("Error opening index")
            return

    """
    Add info send by the index builder to the postings dictionary
    Parameters:
        - doc_id: document id
        - term: term to be added
    """
    def addToPostings(self, doc_id, term):
        if self.builder_postings != {} and term in self.builder_postings.keys():
            if doc_id in self.builder_postings[term].keys():
                self.builder_postings[term][doc_id]+=1
            else:
                self.builder_postings[term][doc_id]=1
        else:
            self.builder_postings[term]={doc_id:1}


### Explicación/documentación

Para implementar un índice con postings, hemos tenido que reconstruir varios de los métodos utilizados por la clase Index para que soporten el uso de postings. El primer paso y más importante ha sido crear un nuevo método addToPostings para poder introducir la información que obtenemos del constructor del índice y transformarla en postings (guardando dos datos para cada término, docuemnto y frecuencia). Sin embargo, para tener mejor acceso a los datos y la estructura de los postings mientras añadimos nuevos desde el builder, addToPostings no utiliza directamente la estructura de tupla que utilizaremos para guardarlos en el índice, sino que los guarda en un diccionario mientras sigamos añadiendo nuevos elementos. Una vez se introduzcan todos los elementos a través del Index Builder, se dará el formato deseado a los datos (en nuestro caso desde el método save).

Hemos modificado también los métodos save y open. El método save está modificado para que haga su funcionalidad anterior, y además guarde los paths (que originalmente en la clase de Index no se guardaban pero es algo necesario) y los postings. Para guardar correctamente los postings les damos formato de tuplas. Guardamos los postings listos para su uso en la variable loaded_postings. El método open ha sido modificado para cargar los postings y los paths, además de heredar la funcionalidad de su clase padre.

Algunos métodos como postings y all_terms también han sido modificados para que funcionen correctamente con la implementación de postings.

## Ejercicio 2.2: Construcción del índice

Implementar la clase RAMIndexBuilder como subclase de Builder, que cree todo el índice en RAM a partir de una colección de documentos.

In [98]:
"""
Class to create a Builder for a Ram Index
"""
class RAMIndexBuilder(Builder):
    """
    Constructor
    Parameters:
        - dir: index directory
    """
    def __init__(self, dir = None):
        super().__init__(dir)
        self.dir = dir
        self.index = RAMIndex()
    
    """
    Commits the index
    """
    def commit(self):
        self.index.save(self.dir)

    """
    Indexes a document
    Parameters:
        - path: the path of the document
        - text: the text of the document
    """
    def index_document(self, path, text):
        doc_id = self.index.ndocs()
        self.index.add_doc(path)
        
        for term in BasicParser.parse(text):
            self.index.addToPostings(doc_id, term)


### Explicación/documentación

Para implementar el Index Builder en Ram hemos heredado de la clase Builder. Principalmente dos modificaciones han sido necesarias, primero de todo que al crear el índice sea tipo RAMIndex, y segundo que al indexar el documento en el índice se llame a la función addToPostings explicada en el ejercicio anterior para que se construyan correctamente los postings. Como hemos decidido que toda la funcionalidad se desarroye dentro del índice, el builder no requiere de más modificaciones.

# Ejercicio 3: Índice en disco* (1pt)

Reimplementar los índices definiendo las clases DiskIndex y DiskIndexBuilder de forma que:

*	El índice se siga creando entero en RAM (por ejemplo, usando estructuras similares a las del ejercicio 2).
*	Pero el índice se guarde en disco dato a dato (docIDs, frecuencias, etc.).
*	Al cargar el índice, sólo el diccionario se lee a RAM, y se accede a las listas de postings en disco cuando son necesarias (p.e. en tiempo de consulta).

Se sugiere guardar el diccionario en un fichero y las listas de postings en otro, utilizando los nombres de fichero definidos como variables estáticas en la clase Config.

Observación: se sugiere inicialmente guardar en disco las estructuras de índice en modo texto para poder depurar los programas. Una vez asegurada la corrección de los programas, puede ser más fácil pasar a modo binario o serializable (usando el módulo pickle como en ejercicios previos).

In [99]:
class DiskIndex(Index):
    # Your new code here (exercise 3*) #
    def __init__(self, dir):
        pass
        raise NotImplementedError

class DiskIndexBuilder(Builder):
    # Your new code here (exercise 3*) #
    def __init__(self, dir):
        raise NotImplementedError

### Explicación/documentación

(por hacer)

# Ejercicio 4: Motor de búsqueda proximal* (1pt)

Implementar un método de búsqueda proximal en una clase ProximitySearcher, utilizando las interfaces de índices posicionales. Igual que en los ejercicios anteriores, se sugiere definir esta clase como subclase (directa o indirecta) de Searcher. Para empezar a probar este buscador, se proporciona una implementación de indexación posicional basada en Whoosh (WhooshPositionalIndex).

In [100]:
"""
Class to create a Searcher for an index based on proximity score
"""
class ProximitySearcher(Searcher):
    """
    Constructor
    Parameters:
        - index: the index to be used
        - parser: the parser to be used
    """
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)
    
    """
    Search method
    Parameters:
        - query: the query to be searched
        - cutoff: the number of results to be returned
    Return:
        - ranking
    """
    def search(self, query, cutoff):

        # Construimos un diccionario con cada documento y posiciones de cada termino
        terms = self.parser.parse(query)
        docs = {}
        doc_scores = []
        for term in terms:
            pos_postings = self.index.positional_postings(term)
            for posting in pos_postings:
                doc_id = posting[0]
                positions = posting[1]
                if doc_id not in docs:
                    docs[doc_id] = {}
                docs[doc_id][term] = positions
        
        # Si solo hay un termino, añadimos 1 al score por cada vez que aparece
        if len(terms) == 1:
            scores = {}
            postings = self.index.postings(terms[0])
            for posting in postings:
                if posting[0] not in scores:
                    scores[posting[0]] = 0
                scores[posting[0]] += posting[1]
            ranking = SearchRanking(cutoff)
            for doc in scores:
                ranking.push(doc, scores[doc])
            return ranking
            
        
        # Para cada documento
        for doc in docs:
            secuencies = []
            # Eliminamos los documentos que no tengan todos los terminos
            if len(docs[doc]) != len(terms):
                doc_scores.append((doc, 0))
                continue

            # Creamos un puntero para cada termino
            pointers = {}
            for term in terms:
                pointers[term] = 0
            
            # Escogemos el termino que apunte un puntero con mayor posicion
            y_term = terms[0]
            y = 0
            for term in terms:
                if docs[doc][term][pointers[term]] > y:
                    y = docs[doc][term][pointers[term]]
                    y_term = term
            
            finish = False
            while not finish:
                # Avanzamos los punteros (menos y) mientras el numero al que apuntan sea menor que y
                for term in terms:
                    if term != y_term:
                        while docs[doc][term][pointers[term]] < y:
                            pointers[term] += 1
                            if pointers[term] >= len(docs[doc][term]):
                                finish = True
                                break
                            if docs[doc][term][pointers[term]] > y:
                                pointers[term] -= 1
                                break

                        if finish:
                            break

                if not finish:
                    # x es el puntero que apunta a la posicion mas pequeña
                    x_term = y_term
                    x = y
                    for term in terms:
                        if docs[doc][term][pointers[term]] < x:
                            x = docs[doc][term][pointers[term]]
                            x_term = term
                    
                    # Añadimos x, y a la lista de posibles
                    secuencies.append((x, y))

                    # Si x es el ultimo puntero, terminamos
                    if pointers[x_term] == len(docs[doc][x_term]) - 1:
                        finish = True
                    else:
                        pointers[x_term] += 1
                        y = docs[doc][x_term][pointers[x_term]]
                        y_term = x_term
            
                
            
            # Calculamos el score con las secuencies
            score = 0
            if len(secuencies) == 0:
                doc_scores.append((doc, 0))
                continue

            for sec in secuencies:
                y = sec[1]
                x = sec[0]
                score += 1 / (y - x - len(terms) + 2)
            doc_scores.append((doc, score))
        
        # Ordenamos los documentos con ranking
        ranking = SearchRanking(cutoff)
        for doc in doc_scores:
            ranking.push(doc[0],doc[1])
        return ranking
                


            

### Explicación/documentación

Para hacer el buscador con búsqueda proximal hemos seguido al pie de la letra el algoritmo de búsqueda proximal. Para ello utilizamos punteros que nos permiten recorrer las diferentes posiciones en las que está cada uno de los términos de la búsqueda para cada documento (Cabe destacar que el algoritmo se ejecuta para cada documento para obtener su score y así al final poder elegir los mejores utilizando un ranking). Gracias a estos punteros podemos encontrar las diferentes secuencias en las que aparecen los términos que estamos buscando, y así podemos calcular el score.

Como la búsqueda proximal solo tiene sentido en el caso de que la búsqueda sea de más de un término, hemos decidido que si la búsqueda es de un solo término, se haga un ranking en función de la frecuencia del término en el documento. Esto no es el método más refinado, pero hemos considerado que es la manera más cercana de hacercarnos al diseño de la búsqueda proximal sin buscar proximidad (ya que lo otro que tiene en cuenta además de la proximidad es la frecuencia de los términos).

# Ejercicio 5: Índice posicional* (1pt)

Implementar una variante adicional de índice (como subclase si se considera oportuno) que extienda las estructuras de índices con la inclusión de posiciones en las listas de postings. La implementación incluirá una clase PositionalIndexBuilder para la construcción del índice posicional así como una clase PositionalIndex para proporcionar acceso al mismo.

In [101]:
"""
Class of a Positional Index based on RAM
"""
class PositionalIndex(RAMIndex):
    """
    Constructor
    Parameters:
        - dir: index directory
    """
    def __init__(self, dir=None):
        self.builder_positions = {}
        self.loaded_positions  = {}
        super().__init__(dir)
            



    """
    Returns all positional postings for a term
    Parameters:
        - term: the term to be searched
    Return:
        - list of tuples (docid, positions)
    """
    def positional_postings(self, term):
        return self.loaded_positions[term]
    
    
    """
    Saves the builded postings created by the index builder.
    Builds the loaded_postings dictionary, needed for the search.
    Parameters:
        - dir: the directory where the index will be saved
    """
    def save(self, dir):
        super().save(dir)

        self.loaded_positions  = {}
        for term in self.builder_positions.keys():
            for doc_id in self.builder_positions[term].keys():
                positions = self.builder_positions[term][doc_id]
                if not term in self.loaded_positions:
                    self.loaded_positions[term] = []
                self.loaded_positions[term].append(tuple((doc_id, positions))) 

        p = os.path.join(dir, Config.POSITIONS_FILE)
        try:
            with open(p, 'wb') as f:
                pickle.dump(self.loaded_positions, f) 
        except Exception as e:
            print("Error saving the positions file", e)

    """
    Loads the builded postings created by the index builder.
    Parameters:
        - dir: the directory for the index to be loaded
    """
    def open(self, dir):
        super().open(dir)

        try:
            p = os.path.join(dir, Config.POSITIONS_FILE)
            with open(p, 'rb') as f:
                self.loaded_positions = pickle.load(f) 
        except Exception as e:
            print("Error opening the positions file", e)

    """
    Adds the info send by the index builder to the postings dictionary
    Parameters:
        - doc_id: document id
        - term: term to be added
        - pos: position of the term in the document
    """
    def addToPostings(self, doc_id, term, pos):
        super().addToPostings(doc_id, term)
        if term not in self.builder_positions.keys():
            self.builder_positions[term] = {}
        if doc_id not in self.builder_positions[term]:
            self.builder_positions[term][doc_id] = [] 
        self.builder_positions[term][doc_id].append(pos)


"""
Class to create a Builder for a Positional Index based on RAM
"""
class PositionalIndexBuilder(RAMIndexBuilder):
    """
    Constructor
    Parameters:
        - dir: index directory
    """
    def __init__(self, dir = None):
        super().__init__(dir)
        self.dir = dir
        self.index = PositionalIndex()

    """
    Indexes a document
    Parameters:
        - path: the path of the document
        - text: the text of the document
    """
    def index_document(self, path, text):
        doc_id = self.index.ndocs()
        self.index.add_doc(path)
        for position, term in enumerate(BasicParser.parse(text)):
            self.index.addToPostings(doc_id, term, position)



### Explicación/documentación, indicando además el tipo de índice que se ha implementado y los aspectos que sean destacables

Para implementar el índice posicional, hemos utilizado como base el índice en RAM que hemos creado previamente en el ejercicio 2. Esto nos ha permitido trabajar con una base ya funcional y conocida, a partir de la que hemos podido implementar los nuevos índices posicionales. A partir de RAMIndex, hemos implementado los siguientes cambios:

En el constructor de índice, hemos reimplementado la función index_document para que, además de su funcionalidad previa, envíe al método addToPostings del índice (donde se desarroya la funcionalidad de guardar los datos para los postings, como está explicado en la documentación del ejercicio 2), además de la id del documento y el término de la petición, la posición donde se encuentra ese término en el documento.

Para utilizar la nueva variante del constructor de índice, hemos cambiado la función del índice addToPostings de manera que ahora además de su funcionalidad previa, guarda en un diccionario las localizaciones de cada término. Además, el método save está modificado para que pueda transformar dicho diccionario en tuplas una vez terminada la construcción del índice, y el método open ha sido modificado para poder cargar estas tuplas de postings posicionales otra vez al índice en RAM. 

# Ejercicio 6: PageRank (1pt)

Implementar el algoritmo PageRank en una clase PagerankDocScorer, que permitirá devolver un ranking de los documentos de manera similar a como hace un Searcher (pero sin recibir una consulta). 

Se recomienda, al menos inicialmente, llevar a cabo una implementación con la que los valores de PageRank sumen 1, para ayudar a la validación de la misma. Posteriormente, si se desea, se pueden escalar (o no, a criterio del estudiante) los cálculos omitiendo la división por el número total de páginas en el grafo. Será necesario tratar los nodos sumidero tal como se ha explicado en las clases de teoría.

In [102]:
"""
Class to create a Page Rank Scorer
"""

class PagerankDocScorer():
    """
    Constructor
    Parameters:
        - graphfile: file
        - r: damping factor
        - n_iter: number of iterations
    """
    def __init__(self, graphfile, r, n_iter):
        self.r = r 
        self.n_iter = n_iter

        self.links = {}

        f = open(graphfile, 'r')
        lines = f.readlines()
        nodes = []

        for l in lines:
            link = l.strip().split('\t')
            saliente, entrante = link[0], link[1]

            if saliente not in nodes:
                nodes.append(saliente)
            if entrante not in nodes:
                nodes.append(entrante)

            if saliente in self.links.keys():
                self.links[saliente].append(entrante)
            else:
                self.links[saliente] = [entrante]
        

        for n in nodes:
            if n not in self.links.keys():
                self.links[n] =  nodes


    """
    Returns a ranking of the documents
    Parameters:
        - cutoff: number of documents to return (maximum)
    Returns:
        - Ranking
    """
    def rank(self, cutoff):
        ranking = SearchRanking(cutoff)
        P = {}
        Pnext = {}
        
        for i in self.links.keys():
            P[i] = 1/len(self.links.keys())

        for _ in range(self.n_iter):

            for i in self.links.keys():
                Pnext[i] = self.r/len(self.links.keys())
            
            for i in self.links.keys():
                out = len(self.links[i])
                for j in self.links[i]:
                    Pnext[j] += (1-self.r)*P[i]/out
            
            for i in self.links.keys():
                P[i] = Pnext[i]
        
        for node in P:
            ranking.push(node, P[node])
        
        return ranking

### Explicación/documentación

Para realizar la implementación de PageRank hemos utilizado el algoritmo de PageRank, de modo que al inicializar un objeto de este tipo se aplica el algoritmo, calculando la cantidad de enlaces entrantes y salientes de cada nodo (página), y teniendo en cuenta los sumideros de manera que redirijan a cualquier otro nodo. Así, al ejecutar el método rank, el algoritmo itera sobre los nodos, calculando el PageRank de cada uno de ellos, y devolviendo una lista con los nodos ordenados por PageRank.

# Celda de prueba

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta **collections** en el mismo directorio que este *notebook*. El fichero <u>toys.zip</u> hay que descomprimirlo para indexar las carpetas que contiene. Igualmente, el fichero <u>graphs.zip</u> incluye ficheros (*1k-links.dat*, *toy-graph1.dat*, *toy-graph2.dat*) que se deben descomprimir en la carpeta collections para que esta celda funcione.

In [103]:
import os
import time

def test_collection(collection_paths: list, index_path: str, word: str, queries: list, analyse_performance: bool):
    print("=================================================================")
    print("Testing indices and search on " + str(len(collection_paths)) + " collections")

    # We now test building different implementations of an index
    test_build(WhooshBuilder(index_path + "whoosh"), collection_paths)
    test_build(WhooshForwardBuilder(index_path + "whoosh_fwd"), collection_paths)
    test_build(WhooshPositionalBuilder(index_path + "whoosh_pos"), collection_paths)
    try:
        test_build(RAMIndexBuilder(index_path + "ram"), collection_paths)
    except NotImplementedError:
        print("RAMIndexBuilder still not implemented")
    try:
        test_build(DiskIndexBuilder(index_path + "disk"), collection_paths)
    except NotImplementedError:
        print("DiskIndexBuilder still not implemented")
    try:
        test_build(PositionalIndexBuilder(index_path + "pos"), collection_paths)
    except NotImplementedError:
        print("PositionalIndexBuilder still not implemented")

    def catch_index(func, name, *args, **kwargs):
        try:
            return func(*args, **kwargs)
        except NotImplementedError:
            print(name + " still not implemented (index)")
            return None

    # We now inspect all the implementations
    indices = [
            WhooshIndex(index_path + "whoosh"),
            WhooshForwardIndex(index_path + "whoosh_fwd"), 
            WhooshPositionalIndex(index_path + "whoosh_pos"), 
            catch_index(lambda: RAMIndex(index_path + "ram"), "RAMIndex"),
            catch_index(lambda: DiskIndex(index_path + "disk"), "DiskIndex"),
            catch_index(lambda: PositionalIndex(index_path + "pos"), "PositionalIndex"),
            ]
    for index in indices:
        if index:
            test_read(index, word)

    for query in queries:
        print("------------------------------")
        print("Checking search results for %s" % (query))
        # Whoosh searcher can only work with its own indices
        test_search(WhooshSearcher(index_path + "whoosh"), WhooshIndex(index_path + "whoosh"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_fwd"), WhooshForwardIndex(index_path + "whoosh_fwd"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_pos"), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        try:
            test_search(ProximitySearcher(WhooshPositionalIndex(index_path + "whoosh_pos")), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        except NotImplementedError:
            print("ProximitySearcher still not implemented")
        for index in indices:
            if index:
                # our searchers should work with any other index
                test_search(SlowVSMSearcher(index), index, query, 5)
                try:
                    test_search(TermBasedVSMSearcher(index), index, query, 5)
                except NotImplementedError:
                    print("TermBasedVSMSearcher still not implemented")
                try:
                    test_search(DocBasedVSMSearcher(index), index, query, 5)
                except NotImplementedError:
                    print("DocBasedVSMSearcher still not implemented")
        try:
            test_search(ProximitySearcher(PositionalIndex(index_path + "pos")), PositionalIndex(index_path + "pos"), query, 5)
        except NotImplementedError:
            print("ProximitySearcher or PositionalIndex still not implemented")

    # if we keep the list in memory, there may be problems with accessing the same index twice
    indices = list()

    if analyse_performance:
        # let's analyse index performance
        test_index_performance(collection_paths, index_path)
        # let's analyse search performance
        for query in queries:
            test_search_performance(collection_paths, index_path, query, 5)

def test_build(builder, collections: list):
    stamp = time.time()
    print("Building index with", type(builder))
    for collection in collections:
        print("Collection:", collection)
        # this function should index the received collection and add it to the index
        builder.build(collection)
    # when we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    # more tests
    doc_id = 0
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) + " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " + str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word '" + word + "':", index.doc_freq(word))
    print("    First two documents:", [(doc, freq) for doc, freq in index.postings(word)][0:2])
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search (engine, index, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " with index " + index.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def disk_space(index_path: str) -> int:
    space = 0
    if os.path.isdir(index_path):
        for f in os.listdir(index_path):
            p = os.path.join(index_path, f)
            if os.path.isfile(p):
                space += os.path.getsize(p)
    return space

def test_index_performance (collection_paths: list, base_index_path: str):
    print("----------------------------")
    print("Testing index performance on " + str(collection_paths) + " document collection")

    print("  Build time...")
    start_time = time.time()
    b = WhooshBuilder(base_index_path + "whoosh")
    for collection_path in collection_paths:
        b.build(collection_path)
    b.commit()
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    b = WhooshForwardBuilder(base_index_path + "whoosh_fwd")
    for collection_path in collection_paths:
        b.build(collection_path)
    b.commit()
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    b = WhooshPositionalBuilder(base_index_path + "whoosh_pos")
    for collection_path in collection_paths:
        b.build(collection_path)
    b.commit()
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    try:
        start_time = time.time()
        b = RAMIndexBuilder(base_index_path + "ram")
        for collection_path in collection_paths:
            b.build(collection_path)
        b.commit()
        print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("RAMIndexBuilder still not implemented")
    try:
        start_time = time.time()
        b = DiskIndexBuilder(base_index_path + "disk")
        for collection_path in collection_paths:
            b.build(collection_path)
        b.commit()
        print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("DiskIndexBuilder still not implemented")

    print("  Load time...")
    start_time = time.time()
    WhooshIndex(base_index_path + "whoosh")
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    WhooshForwardIndex(base_index_path + "whoosh_fwd")
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    WhooshPositionalIndex(base_index_path + "whoosh_pos")
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    try:
        start_time = time.time()
        RAMIndex(base_index_path + "ram")
        print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("RAMIndex still not implemented")
    try:
        start_time = time.time()
        DiskIndex(base_index_path + "disk")
        print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("DiskIndex still not implemented")

    print("  Disk space...")
    print("\tWhooshIndex: %s space ---" % (disk_space(base_index_path + "whoosh")))
    print("\tWhooshForwardIndex: %s space ---" % (disk_space(base_index_path + "whoosh_fwd")))
    print("\tWhooshPositionalIndex: %s space ---" % (disk_space(base_index_path + "whoosh_pos")))
    print("\tRAMIndex: %s space ---" % (disk_space(base_index_path + "ram")))
    print("\tDiskIndex: %s space ---" % (disk_space(base_index_path + "disk")))


def test_search_performance (collection_paths: list, base_index_path: str, query: str, cutoff: int):
    print("----------------------------")
    print("Testing search performance on " + str(collection_paths) + " document collection with query: '" + query + "'")
    whoosh_index = WhooshIndex(base_index_path + "whoosh")
    try:
        ram_index = RAMIndex(base_index_path + "ram")
    except NotImplementedError:
        print("RAMIndex still not implemented")
        ram_index = None
    try:
        disk_index = DiskIndex(base_index_path + "disk")
    except NotImplementedError:
        print("DiskIndex still not implemented")
        disk_index = None

    start_time = time.time()
    test_search(WhooshSearcher(base_index_path + "whoosh"), whoosh_index, query, cutoff)
    print("--- Whoosh on Whoosh %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(SlowVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
    print("--- SlowVSM on Whoosh %s seconds ---" % (time.time() - start_time))

    # let's test some combinations of ranking + index implementations
    try:
        start_time = time.time()
        test_search(TermBasedVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
        print("--- TermVSM on Whoosh %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("TermBasedVSMSearcher still not implemented")
    try:
        if ram_index:
            start_time = time.time()
            test_search(TermBasedVSMSearcher(ram_index), ram_index, query, cutoff)
            print("--- TermVSM on RAM %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("TermBasedVSMSearcher still not implemented")
    try:
        if disk_index:
            start_time = time.time()
            test_search(TermBasedVSMSearcher(disk_index), disk_index, query, cutoff)
            print("--- TermVSM on Disk %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("TermBasedVSMSearcher still not implemented")

    try:
        if disk_index:
            start_time = time.time()
            test_search(DocBasedVSMSearcher(disk_index), disk_index, query, cutoff)
            print("--- DocVSM on Disk %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("DocBasedVSMSearcher still not implemented")

def test_pagerank(graphs_root_dir, cutoff):
    print("----------------------------")
    # we separate this function because it cannot work with all the collections
    print("Testing PageRank")
    try:
        start_time = time.time()
        for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph1.dat", 0.5, 50).rank(cutoff):
            print(score, "\t", path)
        print()
        print("--- Pagerank with toy_graph_1 %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("PagerankDocScorer still not implemented")
    try:
        start_time = time.time()
        for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph2.dat", 0.6, 50).rank(cutoff):
            print(score, "\t", path)
        print()
        print("--- Pagerank with toy_graph_2 %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("PagerankDocScorer still not implemented")
    try:
        start_time = time.time()
        for path, score in PagerankDocScorer(graphs_root_dir + "1k-links.dat", 0.2, 50).rank(cutoff):
            print(score, "\t", path)
        print()
        print("--- Pagerank with simulated links for doc1k %s seconds ---" % (time.time() - start_time))
    except NotImplementedError:
        print("PagerankDocScorer still not implemented")


index_root_dir = "./index/"
collections_root_dir = "./collections/"


test_collection ([collections_root_dir + "toy1/"], index_root_dir + "toy1/", "cc", ["aa dd", "aa"], False)
test_collection ([collections_root_dir + "toy2/"], index_root_dir + "toy2/", "aa", ["aa cc", "bb aa"], False)
test_collection ([collections_root_dir + "toy1/", collections_root_dir + "toy2/"], index_root_dir + "toys/", "aa", ["aa cc", "bb aa"], False)
test_collection ([collections_root_dir + "urls.txt"], index_root_dir + "urls/", "wikipedia", ["information probability", "probability information", "higher probability"], True)
test_collection ([collections_root_dir + "docs1k.zip"], index_root_dir + "docs1k/", "seat", ["obama family tree"], True)
test_collection ([collections_root_dir + "toy2/", collections_root_dir + "urls.txt", collections_root_dir + "docs1k.zip"], index_root_dir + "three_collections/", "seat", ["obama family tree"], True)
#test_collection ([collections_root_dir + "docs10k.zip"], index_root_dir + "docs10k/", "seat", ["obama family tree"], False)
test_pagerank("./collections/", 5)

Testing indices and search on 1 collections
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/toy1/
Done ( 0.023804903030395508 seconds )

Building index with <class '__main__.WhooshForwardBuilder'>
Collection: ./collections/toy1/
Done ( 0.017559528350830078 seconds )

Building index with <class '__main__.WhooshPositionalBuilder'>
Collection: ./collections/toy1/
Done ( 0.01939225196838379 seconds )

Building index with <class '__main__.RAMIndexBuilder'>
Collection: ./collections/toy1/
Done ( 0.0016922950744628906 seconds )

DiskIndexBuilder still not implemented
Building index with <class '__main__.PositionalIndexBuilder'>
Collection: ./collections/toy1/
Done ( 0.002306699752807617 seconds )

DiskIndex still not implemented (index)
Reading index with <class '__main__.WhooshIndex'>
Collection size: 4
Vocabulary size: 39
  Frequency of word "cc" in document 0 - ./collections/toy1/d1.txt: 2
  Total frequency of word "cc" in the collection: 3.0 occurrences over

### Resumen de coste y rendimiento

Hay que analizar las **diferencias de rendimiento** observadas entre las diferentes implementaciones que se han creado y probado para cada componente.

En concreto, hay que reportar tiempo de indexado, consumo máximo de RAM y espacio en disco al construir el índice, y el tiempo de carga y consumo máximo de RAM al cargar el índice para cada una de las colecciones utilizadas.

Por ejemplo:

|      | Construcción | del | índice | Carga del | índice |
|------|--------------------|-----------------|------------------|-----------------|-----------------|
|      | Tiempo de indexado | Consumo máx RAM | Espacio en disco | Tiempo de carga | Consumo máx RAM |
| toy1 | | | | | |
| toy2 | | | | | |
| toys | | | | | |
| 1K | | | | | |
| 10K | | | | | |
